In [1]:
import pandas as pd 
import os
import requests as rq
from bs4 import BeautifulSoup as bs
import re
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from datetime import datetime
from tqdm import tqdm
import warnings
warnings.filterwarnings("ignore")

**Codigo principal PERU 21**

In [2]:
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 Chrome/114.0.0.0 Safari/537.36"
}

In [3]:
with open("APPI OPENAI.txt") as archivo:
  apikey_openai = archivo.read()
os.environ["OPENAI_API_KEY"] = apikey_openai

In [14]:
distritos_lima = [
    "Ancón",
    "Ate",
    "Barranco",
    "Breña",
    "Carabayllo",
    "Chaclacayo",
    "Chorrillos",
    "Cieneguilla",
    "Comas",
    "El Agustino",
    "Independencia",
    "Jesús María",
    "La Molina",
    "La Victoria",
    "Lince",
    "Los Olivos",
    "Lurigancho",
    "Lurín",
    "Magdalena del Mar",
    "Miraflores",
    "Pachacámac",
    "Pucusana",
    "Pueblo Libre",
    "Puente Piedra",
    "Punta Hermosa",
    "Punta Negra",
    "Rímac",
    "San Bartolo",
    "San Borja",
    "San Isidro",
    "San Juan de Lurigancho",
    "San Juan de Miraflores",
    "San Luis",
    "San Martín de Porres",
    "San Miguel",
    "Santa Anita",
    "Santa María del Mar",
    "Santa Rosa",
    "Santiago de Surco",
    "Surquillo",
    "Villa El Salvador",
    "Villa María del Triunfo"
]


In [15]:
distritos_lima = distritos_lima + [distrito.upper() for distrito in distritos_lima]

In [17]:
departamentos_peru = [
    "Amazonas",
    "Áncash",
    "Apurímac",
    "Arequipa",
    "Ayacucho",
    "Cajamarca",
    "Callao", 
    "Cusco",
    "Huancavelica",
    "Huánuco",
    "Ica",
    "Junín",
    "La Libertad",
    "Lambayeque",
    "Lima",
    "Loreto",
    "Madre de Dios",
    "Moquegua",
    "Pasco",
    "Piura",
    "Puno",
    "San Martín",
    "Tacna",
    "Tumbes",
    "Ucayali"
]


In [18]:
departamentos_peru = departamentos_peru + [departamento.upper() for departamento in departamentos_peru]

In [6]:
prompt_pais = ChatPromptTemplate.from_messages([
    ("system", """Tu tarea es identificar el o los países mencionados en una noticia.
Responde únicamente con una lista de nombres de países:
- Si se menciona un solo país, devuélvelo en una lista con un solo elemento.
- Si se mencionan varios países, devuélvelos todos en una lista.
- Si no se menciona ningún país, devuelve una lista vacía.
No des explicaciones ni añadas texto adicional. Solo responde con la lista de países."""),
    ("human", "{input}")
])


In [ ]:
llm = ChatOpenAI(
    temperature=0,
    model="gpt-4")

def pais_detector(texto: str) -> list:
    respuesta = llm.invoke(prompt_pais.format_messages(input=texto)).content.strip().lower()
    return respuesta

def buscar_ordenados(texto, lista):
    encontrados = []

    for lugar in lista:
        patron = r'\b' + re.escape(lugar) + r'\b'
        match = re.search(patron, texto)
        if match:
            encontrados.append((lugar, match.start()))
    
    # Ordenar por posición en el texto
    encontrados.sort(key=lambda x: x[1])
    
    # Aplicar .title() y eliminar duplicados preservando el orden
    vistos = set()
    resultado = []
    for nombre, _ in encontrados:
        nombre_title = nombre.title()
        if nombre_title not in vistos:
            resultado.append(nombre_title)
            vistos.add(nombre_title)
    
    return resultado

In [8]:
lista_url = []
lista_titulo = []
lista_texto = []
lista_fecha = []
lista_distrito = []
lista_departamento = []
lista_pais = []

In [9]:
for pagina in tqdm(range(1, 100)):  # Puedes aumentar el rango si quieres ir más allá
    url = f'https://peru21.pe/noticias/violencia-contra-la-mujer/{pagina}/'
    try:
        html = rq.get(url,headers=headers).content.decode("utf-8")
        soup = bs(html)
        enlaces_raw = soup.find_all('article',{'role':'article','class':'node node--type-article node--view-mode-teaser-2-col'})[0].find_all('div',{'class':'titulo-teaser-2col'})
        links=  ['https://peru21.pe' + raw.find('a').get('href') for raw in enlaces_raw]
        for noticia in tqdm(links):
            try:
                soup2 = bs(rq.get(noticia, headers=headers).content.decode("utf-8"))
                try:
                    titulo = soup2.find_all("h1")[0].text
                    fecha = "/".join(list((re.findall(r"(\d{2})[/-](\d{2})[/-](\d{4})",soup2.find_all('time')[0].text))[0]))
                    if datetime.strptime(fecha, "%d/%m/%Y") > datetime.strptime('24/05/2024', "%d/%m/%Y"):
                        body = "\n".join([p.get_text(strip=True) for p in soup2.find_all("div", {'class': 'content'})[1].find_all(lambda tag: (tag.name == "p"
                                                                                                                                                and tag.get_text(strip=True)  # excluir <p> vacíos
                                                                                                                                                and (
                                                                                                                                                not tag.attrs  # sin atributos
                                                                                                                                                    or set(tag.attrs).issubset({"data-start", "data-end"})  # excepción permitida
                                                                                                                                                )
                                                                                                                                                and not tag.find_parent("div", attrs={"data-entity-type": "node"})  # excluir padres no deseados
                                                                                                                                            )
                                                                                                                                        )])
                    else:
                        divs = soup2.find_all(lambda tag: (
                                            tag.name == 'div'
                                            and 'clearfix' in tag.get('class', [])
                                            and any(
                                                clase in tag.get('class', [])
                                                for clase in [
                                                'field--name-field-texto',
                                                'field--name-field-entradilla'
                                                ]
                                            )
                                            and not tag.find('mark', class_='hl_yellow')
                                            and not tag.find('h2')
                                            and not any(
                                                a for a in tag.find_all('a')
                                                if (not (a.find(['b', 'strong']) or a.find_parent(['b', 'strong']))
                                                and len(tag.get_text(strip=True)) < 200
                                                )
                                            )
                                            and not any(
                                                'mira:' in (el.get_text(strip=True).lower())
                                                for el in tag.find_all(['b', 'i'])
                                            )
                                        )
                                    )
                        body = "".join("".join([p.text for p in divs]).split('\n'))
                    distrito = buscar_ordenados(body, distritos_lima)
                    departamento = buscar_ordenados(body, departamentos_peru)
                    pais = pais_detector(body)
                    lista_pais.append(pais)
                    lista_url.append(noticia)
                    lista_titulo.append(titulo)
                    lista_texto.append(body)
                    lista_fecha.append(fecha)
                    lista_departamento.append(departamento)
                    lista_distrito.append(distrito)
                except:
                    print(f'Error al encontrar metadata de la noticia: No se encontro metadata: {noticia}\n')
            except Exception as e:
                print(f"Error al procesar noticia: {noticia}\n{e}")
    except Exception as e:
        print(f"Error al cargar la página {pagina}:\n{e}")

  3%|▎         | 3/99 [00:56<29:22, 18.36s/it]

Error al encontrar metadata de la noticia: No se encontro metadata: https://peru21.pe/peru21tv/lavozdel21/ministra-mimp-nancy-tolentino-campana-tambien-es-mi-problema-no-podemos-ser-indiferentes-con-la-violencia-a-las-mujeres-dina-boluarte-noticia/



 16%|█▌        | 16/99 [07:26<41:30, 30.01s/it]

Error al encontrar metadata de la noticia: No se encontro metadata: https://peru21.pe/peru21tv/21noticias/violencia-contra-la-mujer-se-reportaron-tres-casos-de-agresion-en-nueve-dias-video-noticia/



 22%|██▏       | 22/99 [10:20<37:50, 29.48s/it]

...

Error al cargar la página 97:
list index out of range


 99%|█████████▉| 98/99 [23:38<00:01,  1.01s/it]

Error al cargar la página 98:
list index out of range


100%|██████████| 99/99 [23:39<00:00, 14.34s/it]

Error al cargar la página 99:
list index out of range


In [10]:
df = pd.DataFrame({
    'url': lista_url,
    'titulo': lista_titulo,
    'texto': lista_texto,
    'fecha': lista_fecha,
    'distrito': lista_distrito,
    'departamento': lista_departamento,
    'pais': lista_pais
})

In [11]:
df

,url,titulo,texto,fecha,distrito,departamento,pais
0,https://peru21.pe/peru/medios-deberan-tener-fr...,Medios deberán tener 'franja educativa' para a...,"El Congreso aprobó la Ley N° 32351, que modifi...",27/05/2025,[Independencia],[La Libertad],"[""perú""]"
1,https://peru21.pe/peru/chiclayo-detienen-profe...,Chiclayo: detienen a profesor por acosar sexua...,La madre de la menor reportó mensajes y fotogr...,16/04/2025,[],[],"[""perú""]"
2,https://peru21.pe/lima/miraflores-sujeto-que-h...,Miraflores: Sujeto violento que hirió con fuen...,"Jueza Susana Quispe, del Cuarto Juzgado de Inv...",26/03/2025,"[Lima, Miraflores]",[Lima],"[""perú""]"
3,https://peru21.pe/lima/atacante-de-miraflores-...,Atacante de Miraflores también agredió a su ex...,Julio Diego Aurelio Solari Zapata fue denuncia...,24/03/2025,"[Miraflores, San Isidro]",[],"[""perú""]"
4,https://peru21.pe/mundo/la-senal-de-auxilio-gl...,La señal de auxilio global que puede salvar la...,El sencillo gesto de abrir la palma de la mano...,20/02/2025,[],[],"[""canadá"", ""méxico"", ""estados unidos"", ""perú""]"
...,...,...,...,...,...,...,...
494,https://peru21.pe/lima/peru-ocupa-segundo-luga...,El Perú ocupa el segundo lugar en feminicidios...,Informe de la Comisión Económica para América ...,26/08/2019,[],[],"[""perú"", ""colombia"", ""república dominicana"", ""..."
495,https://peru21.pe/espectaculos/reimond-manco-a...,Reimond Manco es acusado por su expareja de ha...,Fiorella Alzamora respondió al jugador del UTC...,26/08/2019,[],[Cajamarca],[]
496,https://peru21.pe/mundo/india-adolescente-cond...,India: Adolescente condenada a lamer escupitaj...,El cadáver desnudo de la joven de 16 años apar...,26/08/2019,[],[],"[""india""]"
497,https://peru21.pe/mundo/bolivia-sujeto-le-arra...,Bolivia: Sujeto le arrancó de un mordisco el l...,La Policía inició una intensa búsqueda para da...,26/08/2019,[],[],"[""bolivia""]"


In [12]:
df.to_excel('Noticias_Correo21.xlsx')